# Correlation Dispersion

Part A: Stock Selection// Webscraping and downloading bhavcopies from 1 April 2022 to 28 April 2022

In [1]:
import urllib
import pandas as pd
import numpy as np
import datetime
from zipfile import ZipFile
import os
import requests
import glob
import opstrat as op
import math
from scipy.stats import norm
from numpy import log as ln
from datetime import datetime,timedelta

In [2]:
#stock selection based on past month so past month dates
DownloadStart = '01/04/2022'
DownloadStop = '28/04/2022'

In [3]:
#getting the list of dates
StartDownload = datetime.strptime(DownloadStart,'%d/%m/%Y')
StopDownload = datetime.strptime(DownloadStop,'%d/%m/%Y')

DateIndex = pd.date_range(StartDownload,StopDownload-timedelta(days=0),freq='d')
print(DateIndex)

DatetimeIndex(['2022-04-01', '2022-04-02', '2022-04-03', '2022-04-04',
               '2022-04-05', '2022-04-06', '2022-04-07', '2022-04-08',
               '2022-04-09', '2022-04-10', '2022-04-11', '2022-04-12',
               '2022-04-13', '2022-04-14', '2022-04-15', '2022-04-16',
               '2022-04-17', '2022-04-18', '2022-04-19', '2022-04-20',
               '2022-04-21', '2022-04-22', '2022-04-23', '2022-04-24',
               '2022-04-25', '2022-04-26', '2022-04-27', '2022-04-28'],
              dtype='datetime64[ns]', freq='D')


In [4]:
#downloading and unzipping the files in one folder

import requests
from zipfile import ZipFile
import os.path
    
#link = "https://www1.nseindia.com/content/historical/EQUITIES/2013/JAN/cm01JAN2013bhav.csv.zip"
link1 = "https://www1.nseindia.com/content/historical/DERIVATIVES/"
joiner = "/"
link2 = "fo"
link3 = "bhav.csv.zip"
path = r"C:\Users\admin\Desktop\7. Correlation-Dispersion\PrevMonthBhavcopy\\"
filenamehelp = "bhav.csv"

for i in range(0,len(DateIndex)):
    try:
        Year = DateIndex[i].strftime('%Y')
        Month = (DateIndex[i].strftime('%b')).upper()
        Day = DateIndex[i].strftime('%d')
        finallink = link1 + Year + joiner + Month + joiner + link2 + Day + Month + Year + link3
        filename = link2 + Day + Month + Year + filenamehelp
        completename = path + filename
        df = pd.read_csv(finallink)
        df.to_csv(completename + ".csv")
    except:
        pass

In [5]:
#combining them all into 1 csv file

files = os.path.join("C:\\Users\\admin\\Desktop\\7. Correlation-Dispersion\\PrevMonthBhavcopy\\","*.csv")
files = glob.glob(files)
newdf = pd.concat(map(pd.read_csv,files),ignore_index=True)
newdf['Date'] = pd.to_datetime(newdf['TIMESTAMP'])


In [6]:
filtered = newdf[(newdf['INSTRUMENT']=='OPTSTK') & (newdf['OPTION_TYP']=="CE")]
filtered.reset_index(inplace = True,drop = True)

In [7]:
grouped = pd.DataFrame()
grouped = filtered.groupby(['SYMBOL'])['CONTRACTS'].sum()
grouped = grouped.to_frame().reset_index()
grouped['Rank'] = grouped.rank(axis = 0,method = 'average',numeric_only=True,na_option='keep',ascending = False,pct=False)

In [8]:
finaldata = grouped[grouped['Rank']<=100]
finaldata.reset_index(inplace = True,drop = True)
finaldata = finaldata.sort_values(by = 'Rank', ascending = True)

display(finaldata)

,SYMBOL,CONTRACTS,Rank
78,RELIANCE,3556306,1.0
41,HDFCBANK,2849985,2.0
53,INFY,1858456,3.0
91,TCS,1605120,4.0
39,HDFC,1418455,5.0
...,...,...,...
49,IGL,110014,96.0
5,ASHOKLEY,108601,97.0
69,NMDC,108364,98.0
10,BAJAJ-AUTO,105166,99.0


# Part 2

In [9]:
nextmonthday1 = '29/04/2022'
nextmonthlastday = '26/05/2022'

In [10]:
nextmonthday1 = datetime.strptime(nextmonthday1,'%d/%m/%Y')
nextmonthlastday = datetime.strptime(nextmonthlastday,'%d/%m/%Y')

DateIndex1 = [nextmonthday1,nextmonthlastday]
print(DateIndex1)

[datetime.datetime(2022, 4, 29, 0, 0), datetime.datetime(2022, 5, 26, 0, 0)]


In [11]:
import requests
from zipfile import ZipFile
import os.path

link1 = "https://www1.nseindia.com/content/historical/DERIVATIVES/"
joiner = "/"
link2 = "fo"
link3 = "bhav.csv.zip"
path = r"C:\Users\admin\Desktop\7. Correlation-Dispersion\NewBhavcopy\\"
filenamehelp = "bhav.csv"

for i in range(0,len(DateIndex1)):
    try:
        Year = DateIndex1[i].strftime('%Y')
        Month = (DateIndex1[i].strftime('%b')).upper()
        Day = DateIndex1[i].strftime('%d')
        finallink = link1 + Year + joiner + Month + joiner + link2 + Day + Month + Year + link3
        filename = link2 + Day + Month + Year + filenamehelp
        completename = path + filename
        df = pd.read_csv(finallink)
        df.to_csv(completename + ".csv")       
    except:
        pass

In [12]:
import glob
import os
import pandas as pd

files = os.path.join("C:\\Users\\admin\\Desktop\\7. Correlation-Dispersion\\NewBhavcopy\\","*.csv")
files = glob.glob(files)
newdf1 = pd.concat(map(pd.read_csv,files),ignore_index=True)
newdf1['Date'] = pd.to_datetime(newdf1['TIMESTAMP'])


In [13]:
datafilter = newdf1[(newdf1['INSTRUMENT']=='OPTSTK') & (newdf1['OPTION_TYP']=="CE") & (newdf1["EXPIRY_DT"]=="26-May-2022") & (newdf1['Date']==nextmonthday1)]
datafilter.reset_index(inplace = True,drop = True)

In [14]:
for i in range (0,len(datafilter)):
    if datafilter.loc[i,'SYMBOL'] in finaldata.values:
        datafilter.loc[i,'Check'] = 1
    else:
        datafilter.loc[i,'Check'] = 0

In [15]:
newdata = datafilter[datafilter['Check']==1]
newdata.reset_index(inplace = True,drop = True)

In [16]:
# Downloading the EQ bhavcopy as on new month day 1 to get the spot price

#link = https://www1.nseindia.com/content/historical/EQUITIES/2022/APR/cm29APR2022bhav.csv.zip
link1 = "https://www1.nseindia.com/content/historical/EQUITIES/"
joiner = "/"
link2 = "cm"
link3 = "bhav.csv.zip"
path = r"C:\Users\admin\Desktop\7. Correlation-Dispersion\EQ\\"
filenamehelp = "bhav.csv"

nextmonthday1 = '29/04/2022'
nextmonthday1 = datetime.strptime(nextmonthday1,'%d/%m/%Y')
Year = nextmonthday1.strftime('%Y')
Month = (nextmonthday1.strftime('%b')).upper()
Day = nextmonthday1.strftime('%d')
finallink = link1 + Year + joiner + Month + joiner + link2 + Day + Month + Year + link3
filename = link2 + Day + Month + Year + filenamehelp
completename = path + filename
df1 = pd.read_csv(finallink)
df1.to_csv(completename + ".csv")
df2 = pd.read_csv(completename + ".csv")


In [17]:
for i in range (0,len(df2)):
    if df2.loc[i,'SYMBOL'] in finaldata.values:
        df2.loc[i,'Check'] = 1
    else:
        df2.loc[i,'Check'] = 0        

In [18]:
filterEQ = df2[(df2['Check']==1) & (df2['SERIES']=='EQ')]
filterEQ.reset_index(inplace = True,drop = True)

In [19]:
newEQ = pd.DataFrame()
for i in range(0,len(filterEQ)):
    newEQ.loc[i,'SYMBOL'] = filterEQ.loc[i,'SYMBOL']
    newEQ.loc[i,'Spot'] = filterEQ.loc[i,'CLOSE']    

In [20]:
finalEQ = newEQ.set_index('SYMBOL').T.to_dict('list')

In [21]:
newdata['Spot'] = newdata['SYMBOL'].map(finalEQ)
newdata['finalspot'] = newdata['Spot'].astype('string')

for i in range (0,len(newdata)):
    newdata.loc[i,'fspot'] = newdata.loc[i,'finalspot'][1:-1]  

In [22]:
riskfreerate = 6
assumedvolatility = 30
datetoday = '29/04/2022'
expirydate = '26/05/2022'
StartDate = datetime.strptime(datetoday,'%d/%m/%Y')
StopDate = datetime.strptime(expirydate,'%d/%m/%Y')
daystoexpiry = int(str(StopDate - StartDate)[0:2])

for i in range(0,len(newdata)):
    k = newdata.loc[i,'STRIKE_PR']
    K = float(k)
    st = newdata.loc[i,'fspot']
    St = float(st)
    AssumedPremium = op.black_scholes(K=K,St=St,r=riskfreerate,t=daystoexpiry,v=assumedvolatility,type='c')
    newdata.loc[i,'AssumedPremium'] = AssumedPremium['value']['option value']   

In [23]:
for i in range(0,len(newdata)):
    newdata.loc[i,'IV'] = ((newdata.loc[i,'CLOSE'])*(30))/newdata.loc[i,'AssumedPremium']    

In [24]:
riskfreerate = 6
datetoday = '29/04/2022'
expirydate = '26/05/2022'
StartDate = datetime.strptime(datetoday,'%d/%m/%Y')
StopDate = datetime.strptime(expirydate,'%d/%m/%Y')
daystoexpiry = int(str(StopDate - StartDate)[0:2])

for i in range(0,len(newdata)):
    k = newdata.loc[i,'STRIKE_PR']
    K = float(k)
    st = newdata.loc[i,'fspot']
    St = float(st)
    v = newdata.loc[i,'IV']
    Delta = op.black_scholes(K=K,St=St,r=riskfreerate,t=daystoexpiry,v=v,type='c')
    newdata.loc[i,'Delta'] = Delta['greeks']['delta']
    newdata.loc[i,'Symbol2'] = newdata.loc[i,'SYMBOL']
    newdata.loc[i,'Delta2'] = newdata.loc[i,'Delta']

In [25]:
for i in range(0,len(newdata)):
    newdata.loc[i,'difference'] = abs(newdata.loc[i,'Delta'] - 0.2)

In [26]:
final = newdata.groupby(['SYMBOL'])['difference'].min()
final1 = final.to_frame()
display(final1)

,difference
SYMBOL,
ACC,0.006163
ADANIENT,0.184170
ADANIPORTS,0.250364
AMBUJACEM,0.070894
APOLLOHOSP,0.165219
...,...
UPL,0.002690
VEDL,0.110404
VOLTAS,0.101368


In [27]:
datamatch = final1.to_dict('index')

In [28]:
newdata['match'] = newdata['Symbol2'].map(datamatch)

for i in range (0,len(newdata)):
    newdata.loc[i,'match1'] = str(newdata['match'][i])
    newdata.loc[i,'difference1'] = str(newdata['difference'][i])
    newdata.loc[i,'matcheddelta'] = newdata.loc[i,'match1'][15:-1]

In [29]:
for i in range(0,len(newdata)):
    if ((newdata['difference1'][i])) == ((newdata['matcheddelta'][i])):
        newdata.loc[i,'1or0'] = 1
    else:
        newdata.loc[i,'1or0'] = 0

In [30]:
fdf = pd.DataFrame()
for i in range(0,len(newdata)):
    if newdata['1or0'][i] == 1:
        fdf.loc[i,'SYMBOL'] = newdata.loc[i,'SYMBOL']
        fdf.loc[i,'Delta'] = newdata.loc[i,'Delta']
        fdf.loc[i,'INSTRUMENT'] = newdata.loc[i,'INSTRUMENT']
        fdf.loc[i,'EXPIRY_DT'] = newdata.loc[i,'EXPIRY_DT']
        fdf.loc[i,'STRIKE_PR'] = newdata.loc[i,'STRIKE_PR']
        fdf.loc[i,'OPTION_TYP'] = newdata.loc[i,'OPTION_TYP']
        fdf.loc[i,'CLOSE'] = newdata.loc[i,'CLOSE']
        fdf.loc[i,'Spot'] = newdata.loc[i,'fspot']
        fdf.loc[i,'IV'] = newdata.loc[i,'IV']
        fdf.loc[i,'Date'] = newdata.loc[i,'Date']
    else:
        continue
fdf.reset_index(inplace = True,drop = True)

# Part 3 - Nifty

In [31]:
nifty_filtered = newdf1[(newdf1['INSTRUMENT']=='OPTIDX') & (newdf1['OPTION_TYP']=="CE") & (newdf1['SYMBOL']=='NIFTY') & (newdf1['EXPIRY_DT']=='26-May-2022') & (newdf1['TIMESTAMP']=='29-APR-2022')]
nifty_filtered.reset_index(inplace = True,drop = True)

In [32]:
path = r"C:\Users\admin\Desktop\7. Correlation-Dispersion\Nifty_data.csv"
nifty_spot = pd.read_csv(path)
nifty_spot['Date1'] = pd.to_datetime(nifty_spot['Date'])

In [33]:
for i in range (0,len(nifty_filtered)):
    nifty_filtered['Spot'] = nifty_spot['Close'][0]

In [34]:
#bsm for nifty
riskfreerate = 6
assumedvolatility = 30
datetoday = '29/04/2022'
expirydate = '26/05/2022'
StartDate = datetime.strptime(datetoday,'%d/%m/%Y')
StopDate = datetime.strptime(expirydate,'%d/%m/%Y')
daystoexpiry = int(str(StopDate - StartDate)[0:2])

for i in range(0,len(nifty_filtered)):
    k = nifty_filtered.loc[i,'STRIKE_PR']
    K = float(k)
    st = nifty_filtered.loc[i,'Spot']
    St = float(st)
    AssumedPremium = op.black_scholes(K=K,St=St,r=riskfreerate,t=daystoexpiry,v=assumedvolatility,type='c')
    nifty_filtered.loc[i,'AssumedPremium'] = AssumedPremium['value']['option value']  

In [35]:
for i in range(0,len(nifty_filtered)):
    nifty_filtered.loc[i,'IV'] = ((nifty_filtered.loc[i,'CLOSE'])*(30))/nifty_filtered.loc[i,'AssumedPremium']

In [36]:
riskfreerate = 6
datetoday = '29/04/2022'
expirydate = '26/05/2022'
StartDate = datetime.strptime(datetoday,'%d/%m/%Y')
StopDate = datetime.strptime(expirydate,'%d/%m/%Y')
daystoexpiry = int(str(StopDate - StartDate)[0:2])

for i in range(0,len(nifty_filtered)):
    k = nifty_filtered.loc[i,'STRIKE_PR']
    K = float(k)
    st = nifty_filtered.loc[i,'Spot']
    St = float(st)
    v = nifty_filtered.loc[i,'IV']
    Delta = op.black_scholes(K=K,St=St,r=riskfreerate,t=daystoexpiry,v=v,type='c')
    nifty_filtered.loc[i,'Delta'] = Delta['greeks']['delta']

In [37]:
for i in range(0,len(nifty_filtered)):
    nifty_filtered.loc[i,'difference'] = abs(nifty_filtered.loc[i,'Delta'] - 0.2)  
nifty_filtered.to_csv('niftyfiltered.csv')

In [38]:
nifty = nifty_filtered[nifty_filtered['difference']==nifty_filtered['difference'].min()]
nifty1 = nifty.drop(['OPEN','HIGH','LOW','OPEN_INT','CHG_IN_OI','TIMESTAMP','SETTLE_PR','CONTRACTS','VAL_INLAKH','Unnamed: 15','AssumedPremium','difference'],axis=1)

In [39]:
df_final = fdf.append(nifty1,ignore_index=True)
df_final = df_final.drop(['Unnamed: 0'],axis=1)

In [40]:
for i in range(0,len(df_final)):
    df_final.loc[i,'fullname'] = (df_final['SYMBOL'][i]+str(df_final['STRIKE_PR'][i]))

In [41]:
link1 = "https://www1.nseindia.com/content/historical/EQUITIES/"
joiner = "/"
link2 = "cm"
link3 = "bhav.csv.zip"
path = r"C:\Users\admin\Desktop\7. Correlation-Dispersion\EQ\\"
filenamehelp = "bhav.csv"

nextmonthday1 = '26/05/2022'
nextmonthday1 = datetime.strptime(nextmonthday1,'%d/%m/%Y')
Year = nextmonthday1.strftime('%Y')
Month = (nextmonthday1.strftime('%b')).upper()
Day = nextmonthday1.strftime('%d')
finallink = link1 + Year + joiner + Month + joiner + link2 + Day + Month + Year + link3
filename = link2 + Day + Month + Year + filenamehelp
completename = path + filename
sell1 = pd.read_csv(finallink)
sell1.to_csv(completename + ".csv")
sell2 = pd.read_csv(completename + ".csv")

In [42]:
sell_close = sell2[((sell2['SERIES']=='EQ'))]
sell_close.reset_index(inplace = True,drop = True)

In [43]:
for i in range (0,len(sell_close)):
    if ((sell_close.loc[i,'SYMBOL'] in finaldata.values) or (sell_close.loc[i,'SYMBOL']=='NIFTY')):
        sell_close.loc[i,'Check'] = 1
    else:
        sell_close.loc[i,'Check'] = 0

In [44]:
selldata = sell_close[sell_close['Check']==1]
selldata.reset_index(inplace = True,drop = True)
selldata = selldata.drop(['Unnamed: 0','SERIES','OPEN','HIGH','LOW','LAST','PREVCLOSE','TOTTRDQTY','TOTTRDVAL','TIMESTAMP','TOTALTRADES','ISIN','Unnamed: 13','Check'],axis=1)

display(selldata)

,SYMBOL,CLOSE
0,ACC,2208.00
1,ADANIENT,2046.40
2,ADANIPORTS,701.50
3,AMBUJACEM,365.40
4,APOLLOHOSP,3662.80
...,...,...
95,UPL,747.45
96,VEDL,313.05
97,VOLTAS,995.85
98,WIPRO,453.35


In [45]:
finalsell = selldata.set_index('SYMBOL').T.to_dict('list')
display(finalsell)

{'ACC': [2208.0],
 'ADANIENT': [2046.4],
 'ADANIPORTS': [701.5],
 'AMBUJACEM': [365.4],
 'APOLLOHOSP': [3662.8],
 'ASHOKLEY': [137.4],
 'ASIANPAINT': [2840.15],
 'AUBANK': [1321.5],
 'AUROPHARMA': [535.85],
 'AXISBANK': [683.25],
 'BAJAJ-AUTO': [3857.35],
 'BAJAJFINSV': [12477.75],
 'BAJFINANCE': [5817.8],
 'BALRAMCHIN': [375.45],
 'BANDHANBNK': [316.35],
 'BANKBARODA': [98.35],
 'BEL': [226.0],
 'BHARTIARTL': [693.65],
 'BHEL': [50.2],
 'BIOCON': [325.4],
 'BPCL': [324.35],
 'BSOFT': [353.7],
 'CHAMBLFERT': [345.75],
 'CHOLAFIN': [638.9],
 'CIPLA': [964.55],
 'COALINDIA': [183.25],
 'DEEPAKNTR': [1843.75],
 'DIVISLAB': [3448.7],
 'DLF': [319.85],
 'DRREDDY': [4304.15],
 'EICHERMOT': [2725.1],
 'ESCORTS': [1609.4],
 'GAIL': [151.2],
 'GNFC': [598.75],
 'GODREJCP': [779.95],
 'GRASIM': [1392.55],
 'HAL': [1763.0],
 'HAVELLS': [1172.75],
 'HCLTECH': [980.7],
 'HDFC': [2287.75],
 'HDFCAMC': [1738.15],
 'HDFCBANK': [1366.7],
 'HDFCLIFE': [578.05],
 'HEROMOTOCO': [2636.45],
 'HINDALCO': [40

In [46]:
df_final['sellprice1'] = df_final['SYMBOL'].map(finalsell)
for i in range(0,len(df_final)):
    df_final.loc[i,'sellprice2'] = str(df_final['sellprice1'][i])
    df_final.loc[i,'sellprice'] = (df_final['sellprice2'][i][1:-1])
    if i == 100:
        df_final.loc[i,'sellprice'] = 16170.15

In [47]:
tradesheet = pd.DataFrame()
for i in range(0,len(df_final)):
    tradesheet.loc[i,'SYMBOL'] = df_final.loc[i,'SYMBOL']
    tradesheet.loc[i,'DELTA'] = df_final.loc[i,'Delta']
    tradesheet.loc[i,'INSTRUMENT'] = df_final.loc[i,'INSTRUMENT']
    tradesheet.loc[i,'EXPIRY_DT'] = df_final.loc[i,'EXPIRY_DT']
    tradesheet.loc[i,'STRIKE_PR'] = df_final.loc[i,'STRIKE_PR']
    tradesheet.loc[i,'ENTRYPRICE'] = df_final.loc[i,'CLOSE'] 
    tradesheet.loc[i,'SPOTPRICE'] = df_final.loc[i,'sellprice']
    #tradesheet.loc[i,'SPOTPRICE1'] = tradesheet.loc[i,'SPOTPRICE']
tradesheet['STRIKE_PR'] = tradesheet['STRIKE_PR'].astype(float)   
tradesheet['ENTRYPRICE'] = tradesheet['ENTRYPRICE'].astype(float)
tradesheet['SPOTPRICE'] = pd.to_numeric(tradesheet['SPOTPRICE'])
#tradesheet['SPOT1'] = tradesheet['SPOTPRICE'].astype(float)
#tradesheet['SPOTPRICE'] = tradesheet['SPOTPRICE'].astype()
for i in range(0,len(df_final)):    
    if (tradesheet.loc[i,'SPOTPRICE']) >= (tradesheet.loc[i,'STRIKE_PR']):
        tradesheet.loc[i,'EXITPRICE'] = (tradesheet.loc[i,'SPOTPRICE']) - (tradesheet.loc[i,'STRIKE_PR'])
    else:
        tradesheet.loc[i,'EXITPRICE'] = 0

### Assuming a capital of Rs. 10 crores.
70% we need to use for the OPTSTK and 30% for the OPTIDX

refer to notebook for proper explanation

In [48]:
path = r"C:\Users\admin\Desktop\7. Correlation-Dispersion\fo_mktlots.csv"
lotsize = pd.read_csv(path)
lotsize = lotsize.iloc[:,1:5]
lotsize = lotsize.drop(['22-Jul','22-Jun'],axis=1)

In [49]:
for i in range(0,len(lotsize)):
    if lotsize.loc[i,'Symbol'] in tradesheet.values:
        lotsize.loc[i,'Check'] = 1
    else:
        lotsize.loc[i,'Check'] = 0

In [50]:
lotsizefilter = lotsize[lotsize['Check']==1]
lotsizefilter.reset_index(inplace = True,drop = True)
lotsizefilter = lotsizefilter.drop(['Check'],axis=1)
display(lotsizefilter)

,Symbol,22-Aug
0,NIFTY,50
1,AXISBANK,1200
2,BALRAMCHIN,1600
3,BPCL,1800
4,GODREJCP,1000
...,...,...
96,GRASIM,475
97,SUNPHARMA,700
98,PVR,407
99,TCS,150


In [51]:
finallot = lotsizefilter.set_index('Symbol').T.to_dict('list')
display(finallot)

{'NIFTY': ['50'],
 'AXISBANK': ['1200'],
 'BALRAMCHIN': ['1600'],
 'BPCL': ['1800'],
 'GODREJCP': ['1000'],
 'HDFCLIFE': ['1100'],
 'ACC': ['250'],
 'HINDALCO': ['1075'],
 'IGL': ['1375'],
 'ASIANPAINT': ['200'],
 'GAIL': ['6100'],
 'ONGC': ['3850'],
 'ADANIENT': ['500'],
 'ADANIPORTS': ['1250'],
 'HEROMOTOCO': ['300'],
 'AMBUJACEM': ['1800'],
 'ASHOKLEY': ['5000'],
 'AUBANK': ['1000'],
 'AUROPHARMA': ['1000'],
 'WIPRO': ['1000'],
 'ZEEL': ['3000'],
 'BAJAJ-AUTO': ['250'],
 'BAJAJFINSV': ['50'],
 'BAJFINANCE': ['125'],
 'KOTAKBANK': ['400'],
 'BANDHANBNK': ['1800'],
 'BANKBARODA': ['5850'],
 'BEL': ['3800'],
 'BHARTIARTL': ['950'],
 'BHEL': ['10500'],
 'BIOCON': ['2300'],
 'BSOFT': ['1300'],
 'PEL': ['275'],
 'CHAMBLFERT': ['1500'],
 'CHOLAFIN': ['1250'],
 'CIPLA': ['650'],
 'COALINDIA': ['4200'],
 'DEEPAKNTR': ['250'],
 'DLF': ['1650'],
 'EICHERMOT': ['350'],
 'APOLLOHOSP': ['125'],
 'GNFC': ['1300'],
 'HAL': ['475'],
 'HAVELLS': ['500'],
 'HDFC': ['300'],
 'HDFCAMC': ['300'],
 'HCLTE

In [56]:
tradesheet['lotsize1'] = tradesheet['SYMBOL'].map(finallot)
for i in range(0,len(tradesheet)):
    tradesheet.loc[i,'lotsize2'] = str(tradesheet['lotsize1'][i])
    tradesheet.loc[i,'LOTSIZE'] = tradesheet['lotsize2'][i][2:-2]
    
tradesheet = tradesheet.drop(['lotsize1','lotsize2'],axis=1)

In [55]:
for i in range(0,len(tradesheet)):
    tradesheet.loc[i,'CONTRACTVALUE_FOR_1LOT'] = tradesheet.loc[i,'ENTRYPRICE']*int(tradesheet.loc[i,'LOTSIZE'])
    tradesheet.loc[i,'20%MARGIN_1LOT'] = 0.20*tradesheet.loc[i,'CONTRACTVALUE_FOR_1LOT']
        
    if tradesheet.loc[i,'INSTRUMENT'] == 'OPTSTK':
        tradesheet.loc[i,'Number_of_Lots'] = round(700000/tradesheet.loc[i,'20%MARGIN_1LOT'])
        tradesheet.loc[i,'B/S'] = 'SHORT'
    else:
        tradesheet.loc[i,'Number_of_Lots'] = (30000000/tradesheet.loc[i,'20%MARGIN_1LOT'])
        tradesheet.loc[i,'B/S'] = 'BUY'

In [54]:
for i in range(0,len(tradesheet)):
    tradesheet.loc[i,'MARGINAMOUNT'] = 0.20*(float(tradesheet.loc[i,'ENTRYPRICE'])*int(tradesheet.loc[i,'LOTSIZE'])*int(tradesheet.loc[i,'Number_of_Lots']))
    tradesheet.loc[i,'ENTRYVALUE'] = float(tradesheet.loc[i,'ENTRYPRICE'])*int(tradesheet.loc[i,'LOTSIZE'])*int(tradesheet.loc[i,'Number_of_Lots'])
    tradesheet.loc[i,'EXITVALUE'] = float(tradesheet.loc[i,'EXITPRICE'])*int(tradesheet.loc[i,'LOTSIZE'])*int(tradesheet.loc[i,'Number_of_Lots'])  
    
    if tradesheet.loc[i,'INSTRUMENT'] == 'OPTSTK':
        tradesheet.loc[i,'P/L'] = tradesheet.loc[i,'ENTRYVALUE'] - tradesheet.loc[i,'EXITVALUE']
    else:
        tradesheet.loc[i,'P/L'] = tradesheet.loc[i,'EXITVALUE'] - tradesheet.loc[i,'ENTRYVALUE'] 
    
display(tradesheet)
print(tradesheet['P/L'].sum())
print(((tradesheet['P/L'].sum())/100000000)*100)

tradesheet.to_csv('tradesheet.csv')

,SYMBOL,DELTA,INSTRUMENT,EXPIRY_DT,STRIKE_PR,ENTRYPRICE,SPOTPRICE,EXITPRICE,LOTSIZE,CONTRACTVALUE_FOR_1LOT,20%MARGIN_1LOT,Number_of_Lots,B/S,MARGINAMOUNT,ENTRYVALUE,EXITVALUE,P/L
0,ACC,0.206163,OPTSTK,26-May-2022,2680.0,7.50,2208.00,0.0,250,1875.0,375.0,1867.000000,SHORT,700125.0,3500625.0,0.0,3500625.0
1,ADANIENT,0.384170,OPTSTK,26-May-2022,2560.0,37.55,2046.40,0.0,500,18775.0,3755.0,186.000000,SHORT,698430.0,3492150.0,0.0,3492150.0
2,ADANIPORTS,0.450364,OPTSTK,26-May-2022,910.0,29.85,701.50,0.0,1250,37312.5,7462.5,94.000000,SHORT,701475.0,3507375.0,0.0,3507375.0
3,AMBUJACEM,0.270894,OPTSTK,26-May-2022,435.0,1.05,365.40,0.0,1800,1890.0,378.0,1852.000000,SHORT,700056.0,3500280.0,0.0,3500280.0
4,APOLLOHOSP,0.365219,OPTSTK,26-May-2022,4900.0,63.30,3662.80,0.0,125,7912.5,1582.5,442.000000,SHORT,699465.0,3497325.0,0.0,3497325.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,VEDL,0.310404,OPTSTK,26-May-2022,450.0,4.40,313.05,0.0,1550,6820.0,1364.0,513.000000,SHORT,699732.0,3498660.0,0.0,3498660.0
97,VOLTAS,0.301368,OPTSTK,26-May-2022,1400.0,11.75,995.85,0.0,500,5875.0,1175.0,596.000000,SHORT,700300.0,3501500.0,0.0,3501500.0
98,WIPRO,0.321135,OPTSTK,26-May-2022,560.0,5.85,453.35,0.0,1000,5850.0,1170.0,598.000000,SHORT,699660.0,3498300.0,0.0,3498300.0
99,ZEEL,0.425166,OPTSTK,26-May-2022,265.0,7.00,229.80,0.0,3000,21000.0,4200.0,167.000000,SHORT,701400.0,3507000.0,0.0,3507000.0


195883288.9
195.8832889
